In [7]:
"""
# Virtual Glasses Try-On Experiments

This notebook contains experiments and testing for the Virtual Glasses Try-On system.

## Setup
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import torch

# Project imports
from config.database_config import DatabaseManager
from image_processing.utils.path_utils import ProjectPaths
from image_processing.utils.image_utils import ImageProcessor
from image_processing.preprocess.preprocess_selfies import SelfiePreprocessor
from image_processing.preprocess.preprocess_glasses import GlassesPreprocessor
from models.hybrid_model import HybridVirtualTryOnModel
from demo.demo_tryon import VirtualTryOnDemo

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("Imports completed")

Imports completed


In [8]:
"""
## Initialize System Components
"""

# Initialize core components
db_manager = DatabaseManager()
paths = ProjectPaths()
image_processor = ImageProcessor()

# Test database connection
if db_manager.test_connection():
    print("Database connection successful")
else:
    print("Database connection failed")

# Get system status
status = {
    'Database connected': db_manager.test_connection(),
    'Project root': str(paths.project_root),
    'Raw selfies dir exists': paths.raw_selfies_dir.exists(),
    'Processed selfies dir exists': paths.processed_selfies_dir.exists(),
    'Raw glasses dir exists': paths.raw_glasses_dir.exists(),
    'Processed glasses dir exists': paths.processed_glasses_dir.exists(),
}

print("\nSystem Status:")
for key, value in status.items():
    print(f"  {key}: {value}")


INFO:image_processing.utils.path_utils:Project root set to: /Users/samikshadhere/MSc ADSA/2nd Semester/Case Study I/Final project/Generative-AI-Based-Virtual-Glasses-Try-On
INFO:config.database_config:✅ Database connection successful
INFO:config.database_config:✅ Database connection successful


Database connection successful

System Status:
  Database connected: True
  Project root: /Users/samikshadhere/MSc ADSA/2nd Semester/Case Study I/Final project/Generative-AI-Based-Virtual-Glasses-Try-On
  Raw selfies dir exists: True
  Processed selfies dir exists: True
  Raw glasses dir exists: True
  Processed glasses dir exists: True


In [9]:
"""
## Data Exploration
"""

# Explore glasses data
try:
    glasses_df = db_manager.get_glasses_data(limit=10)
    print(f"Glasses dataset shape: {glasses_df.shape}")
    print("\nColumns:", list(glasses_df.columns))
    print("\nSample data:")
    display(glasses_df.head())
    
    # Analyze glasses data
    if not glasses_df.empty:
        print(f"\nTotal glasses in database: {len(glasses_df)}")
        print(f"Unique brands: {glasses_df['title'].str.split().str[0].nunique()}")
        
except Exception as e:
    print(f"Error loading glasses data: {e}")

# Explore selfies data
try:
    selfies_df = db_manager.get_selfies_data()
    print(f"\nSelfies dataset shape: {selfies_df.shape}")
    if not selfies_df.empty:
        print("Columns:", list(selfies_df.columns))
        display(selfies_df.head())
    else:
        print("No selfies data found. Run preprocessing first.")
        
except Exception as e:
    print(f"Error loading selfies data: {e}")

INFO:config.database_config:✅ Retrieved 10 glasses records


Glasses dataset shape: (10, 36)

Columns: ['id', 'published', 'title', 'main_image', 'additional_images', 'short_description', 'description', 'brand', 'frame_material', 'frame_type', 'frame_shape', 'progressive_lens', 'sustainable', 'target_group', 'lens_width', 'bridge_width', 'temple_length', 'color1', 'color2', 'search_color1', 'search_color2', 'manufacturer_color_code', 'article_number', 'number2', 'web_master_id', 'ean', 'variant', 'availability_value', 'revenue_value', 'margin', 'freshness', 'freshness_until_date', 'highlight', 'highlight_until_date', 'product_sorting', 'fittingbox_id']

Sample data:


,id,published,title,main_image,additional_images,short_description,description,brand,frame_material,frame_type,...,variant,availability_value,revenue_value,margin,freshness,freshness_until_date,highlight,highlight_until_date,product_sorting,fittingbox_id
0,ca8a573e-3549-46b7-8d9d-8d84f336199f,Nein,Ray-Ban 0RX6448 3086,https://optiker-bode.de/sites/default/files/20...,{https://optiker-bode.de/sites/default/files/2...,NaN,NaN,Ray-Ban,Metall,Vollrand,...,NaN,0.0,0.00,0.5,Nein,2021-03-10,Nein,2021-03-10,5,NaN
1,888e23e3-70c4-4f37-a089-51a46592ae8f,Nein,Ray-Ban 0RX4378V 8172,https://optiker-bode.de/sites/default/files/20...,{https://optiker-bode.de/sites/default/files/2...,NaN,NaN,Ray-Ban,Kunststoff,Vollrand,...,NaN,0.0,0.00,0.5,Nein,2021-03-10,Nein,2021-03-10,5,NaN
2,bb4e654d-08a6-4613-a452-66be573e3fba,Nein,Ray-Ban 0RX6466 2904,https://optiker-bode.de/sites/default/files/20...,{https://optiker-bode.de/sites/default/files/2...,NaN,NaN,Ray-Ban,Metall,Vollrand,...,NaN,0.0,0.00,0.5,Nein,2021-03-10,Nein,2021-03-10,5,NaN
3,b54faeb2-0bfc-4b37-b86b-e12732632507,Ja,Ray-Ban 0RX6494 2904,https://optiker-bode.de/sites/default/files/20...,{https://optiker-bode.de/sites/default/files/2...,NaN,NaN,Ray-Ban,Metall,Vollrand,...,NaN,1.0,0.13,0.5,Nein,2021-03-10,Nein,2021-03-10,38,8056597840729.0
4,bddb5bc9-77aa-423c-8271-f9bf5eb28e1d,Nein,Ray-Ban 0RX1972V 2943,https://optiker-bode.de/sites/default/files/20...,{https://optiker-bode.de/sites/default/files/2...,NaN,NaN,Ray-Ban,Metall,Vollrand,...,NaN,0.0,0.00,0.5,Nein,2021-03-10,Nein,2021-03-10,5,NaN


ERROR:config.database_config:❌ Failed to retrieve selfies data: (psycopg2.errors.UndefinedTable) relation "diffusion.selfies" does not exist
LINE 1: SELECT * FROM diffusion.selfies
                      ^

[SQL: SELECT * FROM diffusion.selfies]
(Background on this error at: https://sqlalche.me/e/20/f405)



Total glasses in database: 10
Unique brands: 2

Selfies dataset shape: (0, 0)
No selfies data found. Run preprocessing first.
